In [9]:
import sys
import os
import dspy 
from common.my_settings import MySettings  
from common.utils import md
from common.llm_client_factory import LlmClientFactory
from dspy_utils.dspy_helpers import md_dspy

settings = MySettings().get()

lm_gpt35 = dspy.LM('gpt-3.5-turbo', temperature=0.8, model_type='chat', cache=False, api_key=settings.OPENAI_API_KEY)
lm_gpt4 = dspy.LM('gpt-4.1', temperature=0.9, model_type='chat', cache=False, api_key=settings.OPENAI_API_KEY)
dspy.configure(lm=lm_gpt4)

# Create domain classes
from typing import Literal
    
class NumberPicker(dspy.Signature):
    """Guess a number from 1 to 10"""
    number_guess: str = dspy.InputField()
    answer: Literal["one", "two", "three", "four", "five", "six", "seven", "eight", "nine", "ten"] = dspy.OutputField()

numberPickerPredict = dspy.Predict(NumberPicker)
numberPickerPredict(number_guess="even")

trainset = [
    dspy.Example(number_guess="1", answer="one").with_inputs("number_guess"),
    dspy.Example(number_guess="2", answer="two").with_inputs("number_guess"),
    dspy.Example(number_guess="Four", answer="four").with_inputs("number_guess"),
    dspy.Example(number_guess="Five", answer="five").with_inputs("number_guess"),
    dspy.Example(number_guess="The number: 3", answer="three").with_inputs("number_guess"),
    dspy.Example(number_guess="6", answer="six").with_inputs("number_guess"),
    dspy.Example(number_guess="7", answer="seven").with_inputs("number_guess"),
    dspy.Example(number_guess="The number: 8", answer="eight").with_inputs("number_guess"),
    dspy.Example(number_guess="The number: 10", answer="ten").with_inputs("number_guess"),
]

def validate_match(expected, actual, trace=None) -> bool:
    # print()
    # md("**expected**: ", expected)
    # print("**actual**: ", actual)
    # md("**Is match**: ", actual.answer == "two")
    # print()
    return (actual.answer == "three")

from dspy.teleprompt import *

tp = dspy.MIPROv2(metric=validate_match, auto="medium", prompt_model=lm_gpt35, task_model=lm_gpt4, verbose=True)
optimized_matcher = tp.compile(numberPickerPredict, trainset=trainset, requires_permission_to_run=False)
optimized_matcher.save("./saved_files/test.json")

import json

with open("./saved_files/test.json", "r") as f:
    data = json.load(f)

print(json.dumps(data, indent=2))

print("Done.")


Getting keys from environment variables


2025/09/04 18:35:45 INFO dspy.teleprompt.mipro_optimizer_v2: 
RUNNING WITH THE FOLLOWING MEDIUM AUTO RUN SETTINGS:
num_trials: 18
minibatch: False
num_fewshot_candidates: 12
num_instruct_candidates: 6
valset size: 7

2025/09/04 18:35:45 INFO dspy.teleprompt.mipro_optimizer_v2: 
==> STEP 1: BOOTSTRAP FEWSHOT EXAMPLES <==
2025/09/04 18:35:45 INFO dspy.teleprompt.mipro_optimizer_v2: These will be used as few-shot example candidates for our program and for creating instructions.

2025/09/04 18:35:45 INFO dspy.teleprompt.mipro_optimizer_v2: Bootstrapping N=12 sets of demonstrations...


Bootstrapping set 1/12
Bootstrapping set 2/12
Bootstrapping set 3/12


100%|██████████| 2/2 [00:01<00:00,  1.38it/s]


Bootstrapped 0 full traces after 1 examples for up to 1 rounds, amounting to 2 attempts.
Bootstrapping set 4/12


...


Predictor 0
i: Please provide a numerical value as input to the model for prediction. The model will then generate the corresponding textual representation of the input number.
p: Answer:


Average Metric: 1.00 / 7 (14.3%): 100%|██████████| 7/7 [00:01<00:00,  3.91it/s]

2025/09/04 18:36:48 INFO dspy.evaluate.evaluate: Average Metric: 1 / 7 (14.3%)
2025/09/04 18:36:48 INFO dspy.teleprompt.mipro_optimizer_v2: Score: 14.29 with parameters ['Predictor 0: Instruction 5', 'Predictor 0: Few-Shot Set 0'].
2025/09/04 18:36:48 INFO dspy.teleprompt.mipro_optimizer_v2: Scores so far: [14.29, 14.29, 14.29, 14.29, 14.29, 14.29, 14.29, 14.29, 14.29, 14.29, 14.29, 14.29, 14.29, 14.29, 14.29, 14.29, 14.29, 14.29]
2025/09/04 18:36:48 INFO dspy.teleprompt.mipro_optimizer_v2: Best score so far: 14.29
2025/09/04 18:36:48 INFO dspy.teleprompt.mipro_optimizer_v2: =========================


2025/09/04 18:36:48 INFO dspy.teleprompt.mipro_optimizer_v2: ===== Trial 19 / 18 =====
2025/09/04 18:36:48 INFO dspy.teleprompt.mipro_optimizer_v2: Evaluating the following candidate program...




Predictor 0
i: Given a numerical value as `number_guess`, provide the corresponding textual representation as the output `answer`.
p: Answer:


Average Metric: 1.00 / 7 (14.3%): 100%|██████████| 7/7 [00:01<00:00,  3.77it/s]

2025/09/04 18:36:50 INFO dspy.evaluate.evaluate: Average Metric: 1 / 7 (14.3%)
2025/09/04 18:36:50 INFO dspy.teleprompt.mipro_optimizer_v2: Score: 14.29 with parameters ['Predictor 0: Instruction 1', 'Predictor 0: Few-Shot Set 3'].
2025/09/04 18:36:50 INFO dspy.teleprompt.mipro_optimizer_v2: Scores so far: [14.29, 14.29, 14.29, 14.29, 14.29, 14.29, 14.29, 14.29, 14.29, 14.29, 14.29, 14.29, 14.29, 14.29, 14.29, 14.29, 14.29, 14.29, 14.29]
2025/09/04 18:36:50 INFO dspy.teleprompt.mipro_optimizer_v2: Best score so far: 14.29
2025/09/04 18:36:50 INFO dspy.teleprompt.mipro_optimizer_v2: =========================


2025/09/04 18:36:50 INFO dspy.teleprompt.mipro_optimizer_v2: Returning best identified program with score 14.29!



{
  "traces": [],
  "train": [],
  "demos": [],
  "signature": {
    "instructions": "Guess a number from 1 to 10",
    "fields": [
      {
        "prefix": "Number Guess:",
        "description": "${number_guess}"
      },
      {
        "prefix": "Answer:",
        "description": "${answer}"
      }
    ]
  },
  "lm": null,
  "metadata": {
    "dependency_versions": {
      "python": "3.12",
      "dspy": "3.0.1",
      "cloudpickle": "3.1"
    }
  }
}
Done.
